In [1]:
from __future__ import print_function
from fenics import *  # Importa FEniCS, una biblioteca popular para elementos finitos
import numpy as np  # NumPy para operaciones numéricas
import matplotlib.pyplot as plt  # Matplotlib para gráficos
import math
import scipy

np.set_printoptions(formatter={'float': '{: 0.2f}'.format})
T = 2.0            # final time
num_steps = 50     # number of time steps
dt = T / num_steps # time step size

# Create mesh and define function space
nx = ny = 20
mesh = RectangleMesh(Point(-2, -2), Point(2, 2), nx, ny)
V = FunctionSpace(mesh, 'P', 1)

# Define boundary condition
def boundary(x, on_boundary):
    return on_boundary

bc = DirichletBC(V, Constant(0), boundary)

# Define initial value
u_0 = Expression('exp(-a*pow(x[0], 2) - a*pow(x[1], 2))',
                 degree=2, a=5)
u_n = interpolate(u_0, V)  # Interpola u_D en el espacio de funciones V
u_nn = interpolate(u_0, V) 
u_nnn = interpolate(u_0, V) 
# Definición del problema variacional
u = TrialFunction(V)  # Función de prueba
v = TestFunction(V)   # Función de test

# Formulación del problema variacional
#pconst=[3./2,-2,1./2,0.0] #bdf2
#pconst = [0.48*11/6+0.52*3/2,0.48*-3+0.52*-2,0.48*3/2+0.52*1/2,0.48*-1/3] #bdf2 op
#pconst= [11/6,-3,3/2,-1/3] #bdf 3
pconst=[1,-1,0,0] #bdf1

du=pconst[0]*u
du_n=pconst[1]*u_n
du_nn=pconst[2]*u_nn
du_nnn=pconst[3]*u_nnn
du_t= du+du_n +du_nn +du_nnn
f = Constant(0)

F = du_t*v*dx  + dt*dot(grad(u), grad(v))*dx 
a, L = lhs(F), rhs(F)

# Create VTK file for saving solution
vtkfile_u = XDMFFile("results/u_fem.xdmf")
vtkfile_u.parameters["flush_output"] = True
vtkfile_u.parameters["rewrite_function_mesh"] = False

# Time-stepping
u_ = Function(V)
t = 0
for n in range(num_steps):

    # Update current time
    t += dt

    # Compute solution
    solve(a == L, u_, bc)
    # Update previous solution
    u_n.assign(u_)
    u_.rename("u_a", "u_a");vtkfile_u.write(u_, t)




Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational p

In [5]:
K_fem =dot(grad(u), grad(v))*dx  # Formulación débil
C_fem=u*v*dx

# algoritmo de arnoldi 
def arnoldi_iteration_3(A, b, n):
    m = A.shape[0]

    h = np.zeros((n + 1, n))
    Q = np.zeros((m, n + 1))

    q = b / np.linalg.norm(b)
    Q[:, 0] = q

    for k in range(n):
        v = A.dot(q)
        for j in range(k + 1):
            h[j, k] = np.dot(Q[:, j].conj(), v)  # <-- Q needs conjugation!
            v = v - h[j, k] * Q[:, j]

        h[k + 1, k] = np.linalg.norm(v)
        eps = 1e-12
        if h[k + 1, k] > eps:
            q = v / h[k + 1, k]
            Q[:, k + 1] = q
        else:
            return Q[:,0:n], h[0:n, 0:n]
    return Q[:,0:n], h[0:n, 0:n]

# proceso 
vtkfile_u = XDMFFile("results/u_exp.xdmf")
vtkfile_u.parameters["flush_output"] = True
vtkfile_u.parameters["rewrite_function_mesh"] = False

t= 0
u_=Function(V)

K= assemble(lhs(K_fem))# Separa la parte izquierda y derecha de la ecuación
bc.apply(K)
K_=K.array()
C=assemble(C_fem)
bc.apply(C)
C_1=np.linalg.inv(C.array())
A=-np.dot(C_1,K_)

u_n = interpolate(u_0, V)
u_i=u_n.vector().get_local()
m=30
v_hat=np.zeros(m)
v_hat[0]=1

for n in range(num_steps):
    t += dt
    
    
    print(f'step:{n} of {num_steps} time= {t}')
    
    Beta=np.linalg.norm(u_i)
    #V_m,H_m = arnoldi_iteration_3(A,u_i,m)
    #u_i=Beta*np.dot(np.dot(V_m,scipy.linalg.expm(dt*H_m)),v_hat.T)
    u_i=np.dot(scipy.linalg.expm(dt*A),u_i)
    u_.vector()[:]=u_i
    
    print(Beta)
    u_i=u_.vector().get_local()
    u_.rename("u_a", "u_a");vtkfile_u.write(u_, t)


step:0 of 50 time= 0.04
2.802495608306806
step:1 of 50 time= 0.08
2.0680995894894347
step:2 of 50 time= 0.12
1.7216040001810486
step:3 of 50 time= 0.16
1.5072235630022008
step:4 of 50 time= 0.2
1.3573641297494758
step:5 of 50 time= 0.24000000000000002
1.2445023699959696
step:6 of 50 time= 0.28
1.154764967048595
step:7 of 50 time= 0.32
1.0802549248762552
step:8 of 50 time= 0.36
1.0161726918689309
step:9 of 50 time= 0.39999999999999997
0.9594991330029043
step:10 of 50 time= 0.43999999999999995
0.908296614483666
step:11 of 50 time= 0.4799999999999999
0.8613024457336461
step:12 of 50 time= 0.5199999999999999
0.8176806930109921
step:13 of 50 time= 0.5599999999999999
0.7768671588195759
step:14 of 50 time= 0.6
0.7384715501936037
step:15 of 50 time= 0.64
0.7022154546666222
step:16 of 50 time= 0.68
0.6678929309929319
step:17 of 50 time= 0.7200000000000001
0.6353454331352588
step:18 of 50 time= 0.7600000000000001
0.6044458309819074
step:19 of 50 time= 0.8000000000000002
0.5750882072168744
step:2